# Kuramoto-Shinomoto-Sakaguchi MV-SDE

Importiamo, in primis, i pacchetti necessari per usarele funzioni matematiche in python

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jan 19 12:13:37 2023
@author: Andrea
"""
import math
import numpy as np 
import numpy.matlib as ml

Ricordiamo a cosa corrisponderanno i valari che prenderemo in input nelle funzioni che definiremo
* N : numero di iterazioni (steps temporali)
* M : numero di simulazioni in ogni istante
* T : istante finale
* $\mu$ : funzione di Drift
* $\sigma$ : funzione di Diffuzione

Riscriviamo infine l'equazione di Kuramoto-Shninomoto-Sakaguchi forma differenziale

$ dX_t = \left( c_1 sen(X_t) - c_2 cos(X_t) \right) dt + \sigma dW_t $

Siamo pronti per definire la prima funzione che applica il Metodo di Eulero a un vettore X preso in input, assieme alle costanti $c_1$ e $c_2$, il valore della parte diffusiva $\sigma$ , il numero di simulazioni M e il valore dello step temporale h

In [4]:
def eulero(c1, c2, sigma, h, M, X):
    W = np.random.normal(0, 1, (1, M)) 
    drift = c1 * np.cos(X) - c2 * np.sin(X) 
    diffusione = sigma 
    X = X + drift * h + diffusione * math.sqrt(h) * W 
    
    return X

Ora definiamo la funzione volta ad applicare il metodo Monte Carlo, che banalmente fa la media degli M valori di un vettore preso in input 

In [5]:
def monte_carlo(M, Y):
    X = sum(Y) / M
    return X

Concludiamo riportanto il main che richiama le due funzioni sopra definite. Notiamo come il Metodo di Eulero vada applicato ciclicamente per ogni step temporale affinchè sia valido questo algoritmo.

In [2]:
if __name__ == "__main__":
    
    # Parametri in Input
    c1 = 1
    c2 = 0
    sigma = 0.5
    T = 10
    N = 2
    M = 10
    X0 = 100
    
    # Richiamo funzioni
    h = T / N
    X = ml.repmat(X0, 1, M)
    
    for i in range(N):
        XT = eulero(c1, c2, sigma, h, M, X)
        
        YT = c1 * np.sin(XT) + c2 * np.cos(XT)
        YT = YT.transpose()
        F = monte_carlo(M, YT)
    print(XT)
    print(F)

[[103.58429402 104.98558868 102.54015373 105.29593541 104.22888701
  105.64113338 103.25492295 103.44831381 102.66763293 104.77735684]]
[-0.18432758]
